# EcoVoyage Travel — Agent Notebook
This notebook demonstrates a tool-enabled chatbot for a fictional business.

In [1]:
# Optional: install deps in Colab/Notebook
# !pip install -r business_bot/requirements.txt

## Download business_bot from Google Drive (public)
If running in Colab or a clean notebook, fetch the project folder from a public Drive link into `business_bot/`.

In [2]:
import os, sys, subprocess, shutil
from pathlib import Path

ROOT_DIR = Path.cwd()
TARGET = ROOT_DIR / 'business_bot'
DRIVE_URL = os.environ.get('GDRIVE_FOLDER_URL', 'https://drive.google.com/drive/folders/1mY3PsShM1nc_WciI2CyXKXm373K0SRiH?usp=sharing')

if not TARGET.exists():
    # Install gdown quietly
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'gdown'], check=True)
    import gdown
    print('Downloading business_bot from Google Drive to', TARGET)
    gdown.download_folder(DRIVE_URL, output=str(TARGET), quiet=False, use_cookies=False)
    # If a single subfolder was created, lift its contents up to business_bot/
    entries = [p for p in TARGET.iterdir()]
    if len(entries) == 1 and entries[0].is_dir():
        inner = entries[0]
        for item in inner.iterdir():
            shutil.move(str(item), TARGET / item.name)
        inner.rmdir()
    print('Ready:', list(TARGET.iterdir()))


In [3]:
# Optional: install deps in Colab/Notebook
!pip install -r business_bot/requirements.txt

In [4]:
import os, json
from pathlib import Path
from dotenv import load_dotenv
from pdfminer.high_level import extract_text
from openai import OpenAI

from business_bot.tools import record_customer_interest, record_feedback, openai_tools

ROOT = Path('business_bot')
SUMMARY_PATH = ROOT / 'business_summary.txt'
PDF_PATH = ROOT / 'about_business.pdf'

def load_context():
    summary = SUMMARY_PATH.read_text(encoding='utf-8') if SUMMARY_PATH.exists() else ''
    pdf_text = extract_text(str(PDF_PATH)) if PDF_PATH.exists() else ''
    return {'summary': summary, 'pdf': pdf_text}

SYSTEM_PROMPT = ("You are EcoVoyage Travel's virtual concierge. Use the provided context to answer.\n\n"
                 "If unsure, call record_feedback. Encourage leads to share name/email and call record_customer_interest.")

def build_initial_messages(context):
    return [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'system', 'content': 'Business summary:' + context.get('summary','')},
        {'role': 'system', 'content': 'Business PDF:' + context.get('pdf','')},
    ]

def chat_openai(messages, tools, model='gpt-4o-mini'):
    load_dotenv()
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    while True:
        resp = client.chat.completions.create(model=model, messages=messages, tools=tools, tool_choice='auto')
        msg = resp.choices[0].message
        if msg.tool_calls:
            # Include assistant message with tool_calls before appending tool outputs
            assistant_tool_calls = []
            for tc in msg.tool_calls:
                assistant_tool_calls.append({
                    'id': tc.id,
                    'type': 'function',
                    'function': {
                        'name': tc.function.name,
                        'arguments': tc.function.arguments,
                    },
                })
            messages.append({'role': 'assistant', 'content': msg.content or '', 'tool_calls': assistant_tool_calls})

            for tc in msg.tool_calls:
                fn = tc.function
                args = json.loads(fn.arguments) if isinstance(fn.arguments, str) else (fn.arguments or {})
                if fn.name == 'record_customer_interest':
                    out = record_customer_interest(args.get('email',''), args.get('name',''), args.get('message',''))
                elif fn.name == 'record_feedback':
                    out = record_feedback(args.get('question',''))
                else:
                    out = 'Unknown tool'
                messages.append({'role': 'tool', 'tool_call_id': tc.id, 'name': fn.name, 'content': out})
            continue
        return msg.content


In [18]:
# Demo (requires valid OPENAI_API_KEY in .env)
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

ctx = load_context()
messages = build_initial_messages(ctx)
messages.append({'role':'user','content':'Hi! Do you offer family-friendly trips in Costa Rica?'})
chat_openai(messages, openai_tools)

"Yes, we do offer family-friendly trips in Costa Rica! Our itineraries can be tailored to suit families, including activities that are engaging for all ages while ensuring a safe and eco-friendly experience. \n\nIf you're interested in more details or want to explore specific options, please share your name and email so I can follow up with you!"

## Launch Gradio (optional)
Run a simple chat UI inside the notebook.

In [19]:
# Demo (requires valid OPENAI_API_KEY in .env)
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')


import gradio as gr

def nb_gradio_chat(user_input, history):
    ctx = load_context()
    messages = build_initial_messages(ctx)
    for u,a in history:
        if u: messages.append({'role':'user','content':u})
        if a: messages.append({'role':'assistant','content':a})
    messages.append({'role':'user','content':user_input})
    return chat_openai(messages, openai_tools)

iface = gr.ChatInterface(fn=nb_gradio_chat, title='EcoVoyage Travel Concierge (Notebook)')
iface.launch()  # Uncomment to launch in supported notebook environments


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b59705044d53facbc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
